In [1]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

#Nomalization
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

# Show the image
def imshow(img):
    img = img / 2 + 0.5     
    # Take the transpose
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

Files already downloaded and verified
Files already downloaded and verified


In [2]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5, padding=2)
        #pooling 16 
        self.conv2 = nn.Conv2d(32, 96, 3, padding=1)
        #pooling 8 
        self.conv3 = nn.Conv2d(96, 192, 3, padding=1)
        # 8
        self.conv4 = nn.Conv2d(192, 128, 3, padding=1)
        # 8
        self.conv5 = nn.Conv2d(128, 128, 3)
        #pooling 3
        self.fc1 = nn.Linear(128 * 3 * 3, 2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.fc3 = nn.Linear(2048, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(F.relu(self.conv5(x)), (2, 2))
        x = x.view(x.size(0), 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net1()
net = net.cuda()
print(net)

Net1 (
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear (1152 -> 2048)
  (fc2): Linear (2048 -> 2048)
  (fc3): Linear (2048 -> 10)
)


In [3]:
# Cross Entroy Loss Function
criterion = nn.CrossEntropyLoss()
# Optimizer：SGD
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [4]:
# loop over epoches
for epoch in range(4):  

    running_loss = 0.0
# enumerate function in python to enumerate(one batch a time)
    for i, data in enumerate(trainloader, 0):
        # input
        inputs, labels = data
        
        # put in Variable(pytorch)
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        # Set Variables' Gradient as Zeros
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.303
[1,  4000] loss: 2.195
[1,  6000] loss: 1.894
[1,  8000] loss: 1.739
[1, 10000] loss: 1.618
[1, 12000] loss: 1.522
[2,  2000] loss: 1.413
[2,  4000] loss: 1.364
[2,  6000] loss: 1.291
[2,  8000] loss: 1.225
[2, 10000] loss: 1.173
[2, 12000] loss: 1.124
[3,  2000] loss: 1.029
[3,  4000] loss: 0.982
[3,  6000] loss: 0.954
[3,  8000] loss: 0.937
[3, 10000] loss: 0.907
[3, 12000] loss: 0.882
[4,  2000] loss: 0.785
[4,  4000] loss: 0.798
[4,  6000] loss: 0.767
[4,  8000] loss: 0.749
[4, 10000] loss: 0.775
[4, 12000] loss: 0.724
Finished Training


In [109]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    labels = labels.cuda()
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 76 %


In [110]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    labels = labels.cuda()
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print(' %5s Accuarcy of Prediction: %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

    飞机 图片预测的准确度: 72 %
    汽车 图片预测的准确度: 85 %
     鸟 图片预测的准确度: 68 %
     猫 图片预测的准确度: 54 %
     鹿 图片预测的准确度: 78 %
     狗 图片预测的准确度: 71 %
    青蛙 图片预测的准确度: 81 %
     马 图片预测的准确度: 77 %
     船 图片预测的准确度: 90 %
    卡车 图片预测的准确度: 86 %
